In [2]:
#!python

from typing import Tuple
import flow
import signac
import hoomd
import gsd.hoomd
import freud
import numpy as np
import pandas as pd
import polars as pl

import glob
import os
import pickle
import copy
import gc
import shutil
import multiprocessing
import pathlib
import math
import itertools

from tqdm import tqdm

from dataclasses import dataclass
from collections import defaultdict

from monk import workflow, methods, pair, utils, prep
import schmeud._schmeud as _schmeud
from schmeud import ml

# from excess_entropy import *


In [3]:
def _setup_nvt_sim(job: signac.Project.Job, sim: hoomd.Simulation, temp=None):
    doc = job.doc
    sp = job.sp
    project = job._project
    proj_doc = project.doc
    dt = proj_doc["dt"]

    pot = sp["pot"]
    if isinstance(pot, str):
        pot = [pot]
    elif isinstance(pot, list):
        pass
    else:
        raise ValueError("The 'potential' must be a string or list")

    integrator = hoomd.md.Integrator(dt=dt)
    pair_func, args = prep.search_for_pair(pot)
    tree = hoomd.md.nlist.Tree(0.3)
    print("Using pair function: ", pair_func)
    force = pair_func(tree, *args)

    if temp is None:
        temp = doc["init_temp"]
    nvt = hoomd.md.methods.NVT(hoomd.filter.All(), temp, dt*100)
    integrator.forces = [force]
    integrator.methods = [nvt]

    sim.operations.integrator = integrator

    return nvt

In [ ]:
sim = prep.quick_sim(sp["N"], 1.2, hoomd.device.GPU(), dim=2,
                    ratios=[60, 40], diams=[1.0, 0.88], 0)
nvt = _setup_nvt_sim(job, sim)

sim.run(0)
# sim.state.thermalize_particle_momenta(filter=hoomd.filter.All(), kT=doc["init_temp"])
print("Running initial equilibration")

sim.run(equil_steps)

# hoomd.write.GSD.write(sim.state, job.fn("init.gsd"))